In [1]:
import numpy as np
import pandas as pd

from collections import defaultdict
from collections import Counter

import os

In [2]:
data_dir = "../../../data/"

old_folder = "intermediate_data/hla_i_leave1out_match_raw_v/"
new_folder = "HLA_I_leave1out_match/"

In [3]:
file_names = os.listdir(data_dir + old_folder)

In [4]:
# load the translate file to make translate dict

df_v_allele_trans = \
pd.read_csv(data_dir + "intermediate_data/v_allele_translate_table_fillin.csv", header=0)

df_v_allele_trans.shape

(65, 2)

In [5]:
v_allele_trans_dict = defaultdict(str)
for v_allele, translate in \
        zip(df_v_allele_trans.v_allele_new.tolist(),
            df_v_allele_trans.v_allele_translate.tolist()):
    v_allele_trans_dict[v_allele] = translate

len(v_allele_trans_dict)

65

In [6]:
list(v_allele_trans_dict.values())

['TRBV1*01',
 'TRBV2*01',
 'TRBV3-1*01',
 'TRBV3-2*01',
 'TRBV3-2*02',
 'TRBV4-1*01',
 'TRBV4-2*01',
 'TRBV4-3*01',
 'TRBV5-1*01',
 'not_found',
 'TRBV5-3*01',
 'TRBV5-4*01',
 'TRBV5-5*01',
 'TRBV5-6*01',
 'TRBV5-7*01',
 'TRBV5-8*01',
 'TRBV6-1*01',
 'TRBV6-4*01',
 'TRBV6-5*01',
 'TRBV6-6*01',
 'TRBV6-7*01',
 'TRBV6-8*01',
 'TRBV6-9*01',
 'TRBV7-1*01',
 'TRBV7-2*01',
 'TRBV7-3*01',
 'TRBV7-3*02',
 'TRBV7-3*03',
 'TRBV7-4*01',
 'not_found',
 'not_found',
 'TRBV7-6*01',
 'TRBV7-7*01',
 'TRBV7-8*01',
 'TRBV7-8*02',
 'TRBV7-9*01',
 'not_found',
 'TRBV9*01',
 'TRBV10-1*01',
 'TRBV10-2*01',
 'TRBV10-3*01',
 'TRBV11-1*01',
 'TRBV11-2*02',
 'TRBV11-3*01',
 'TRBV12-1*01',
 'TRBV12-2*01',
 'TRBV12-5*01',
 'TRBV13*01',
 'TRBV14*01',
 'TRBV15*01',
 'TRBV16*01',
 'TRBV17*01',
 'TRBV18*01',
 'TRBV19*01',
 'TRBV20-1*01',
 'TRBV21-1*01',
 'TRBV23-1*01',
 'TRBV25-1*01',
 'TRBV26*01',
 'TRBV27*01',
 'TRBV28*01',
 'TRBV29-1*01',
 'TRBV30*01',
 'TRBV30*02',
 'not_found']

In [7]:
os.makedirs(data_dir+new_folder+"HLA_I_leave1out_match_B0801/test")
os.makedirs(data_dir+new_folder+"HLA_I_leave1out_match_B0801/train_valid")

os.makedirs(data_dir+new_folder+"HLA_I_leave1out_match_B0702/test")
os.makedirs(data_dir+new_folder+"HLA_I_leave1out_match_B0702/train_valid")

os.makedirs(data_dir+new_folder+"HLA_I_leave1out_match_C0701/test")
os.makedirs(data_dir+new_folder+"HLA_I_leave1out_match_C0701/train_valid")

In [8]:
index_dict = defaultdict(str)

index_dict["1"] = "B0801"
index_dict["11"] = "B0702"
index_dict["23"] = "C0701"

In [9]:
for cur_file in file_names:
    
    cur_df = pd.read_csv(data_dir + old_folder + cur_file, header=0)
    
    cur_tcr_old = cur_df.tcr.tolist()
    cur_hla = cur_df.hla_allele.tolist()
    
    cur_tcr_new = [",".join([v_allele_trans_dict[x.split(",")[0]], x.split(",")[1]]) for x in cur_tcr_old]
    
    trans_df = pd.DataFrame(list(zip(cur_tcr_new, cur_hla)), columns = ["tcr", "hla_allele"])
    
    new_filename = "_".join(cur_file.split("_")[(-3):(-1)])
    
    subfolder_name = "HLA_I_leave1out_match_"+index_dict[cur_file.split(".")[0].split("_")[-1]]
    
    if cur_file.split("_")[-3] == "test":
        trans_df.to_csv(data_dir+new_folder+ subfolder_name+"/test/" + new_filename, 
                        index = False)
    else:
        trans_df.to_csv(data_dir+new_folder+ subfolder_name+"/train_valid/" + new_filename, 
                        index = False)        

In [11]:
# combine all data files together
# to (1)verify whether all translated v alleles belong to those in combo_xcr.tsv
# (2)get the maximum length of CDR3s involved

all_tcrs = []
all_hlas = []

for cur_file in file_names:

    new_filename = "_".join(cur_file.split("_")[(-3):(-1)])
    
    subfolder_name = "HLA_I_leave1out_match_"+index_dict[cur_file.split(".")[0].split("_")[-1]]
    
    if cur_file.split("_")[-3] == "test":
        cur_df = pd.read_csv(data_dir+new_folder+ subfolder_name+"/test/" + new_filename, 
                             header=0)
    else:
        cur_df = pd.read_csv(data_dir+new_folder+ subfolder_name+"/train_valid/" + new_filename, 
                             header=0)        
        
    print(cur_df.shape)

    all_tcrs = all_tcrs + cur_df.tcr.tolist()
    all_hlas = all_hlas + cur_df.hla_allele.tolist()

(878, 2)
(20025, 2)
(530, 2)
(1394, 2)
(3704, 2)
(1234, 2)
(6970, 2)
(4005, 2)
(2650, 2)
(2490, 2)
(4182, 2)
(4390, 2)
(18520, 2)
(6675, 2)
(1335, 2)
(6170, 2)
(498, 2)
(20910, 2)


In [12]:
len(set([x.split(",")[0] for x in all_tcrs]))

44

In [13]:
# verify the overlap between old v alleles and those corresponding to not_found
# after translation

all_tcrs_old = []

for cur_file in file_names:
    
    cur_df = pd.read_csv(data_dir + old_folder + cur_file, header = 0)
    print(cur_df.shape)
    all_tcrs_old = all_tcrs_old + cur_df.tcr.tolist()

(878, 2)
(20025, 2)
(530, 2)
(1394, 2)
(3704, 2)
(1234, 2)
(6970, 2)
(4005, 2)
(2650, 2)
(2490, 2)
(4182, 2)
(4390, 2)
(18520, 2)
(6675, 2)
(1335, 2)
(6170, 2)
(498, 2)
(20910, 2)


In [14]:
old_tcr_set = set([x.split(",")[0] for x in all_tcrs_old])

In [15]:
not_found_set = set([v for v, trans in zip(df_v_allele_trans.v_allele_new.tolist(),
                                           df_v_allele_trans.v_allele_translate.tolist()) if trans == "not_found"])

not_found_set

{'TCRBV05-02*01',
 'TCRBV07-05*01',
 'TCRBV07-05*02',
 'TCRBV08-02*01',
 'TCRBVA-or09_02*01'}

In [18]:
# indeed, there is no old format v gene allele translated into not_found
# among those used in train/validation/test under all_match setting
len(old_tcr_set.intersection(not_found_set))

0

In [19]:
# look into the range of CDR3 length

set([len(x.split(",")[1]) for x in all_tcrs])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}